In [9]:
import sys
sys.path.insert(0,"../../")

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import *
from bs4.element import PageElement
import time
from base import *

folder = ""
host = f'basketball-reference.com'


All Franchises

In [133]:
def parse_all_franchises(soup):
    teams_active = parse_table(soup,'#teams_active tbody tr')
    teams_active = teams_active[teams_active['franch_name'].notna()]
    teams_active['active'] = True
    teams_defunct = parse_table(soup,'#teams_defunct tbody tr')
    teams_defunct = teams_defunct[teams_defunct['franch_name'].notna()]
    teams_defunct['active'] = False
    teams_all = pd.concat([teams_active,teams_defunct],ignore_index=True)
    return teams_all

url = f'basketball-reference.com/teams/'
soup = BeautifulSoup(fetch_html(f"{url}"))
all_franchises = parse_all_franchises(soup)

if not os.path.isdir(f"{url}"): 
    os.makedirs(f"{url}")
# all_franchises.to_csv(f"{url}/all_franchises.csv")
all_franchises

,franch_name,franch_name_href,lg_id,year_min,year_max,years,g,wins,losses,win_loss_pct,years_playoffs,years_division_champion,years_conference_champion,years_league_champion,team_name,active
0,Atlanta Hawks,/teams/ATL/,NBA,1949-50,2022-23,74,5809,2867,2942,.494,48,12,0,1,NaN,True
1,Boston Celtics,/teams/BOS/,NBA/BAA,1946-47,2022-23,77,5986,3539,2447,.591,59,32,10,17,NaN,True
2,Brooklyn Nets,/teams/NJN/,NBA/ABA,1967-68,2022-23,56,4483,1974,2509,.440,30,5,2,2,NaN,True
3,Charlotte Hornets,/teams/CHA/,NBA,1988-89,2022-23,33,2585,1136,1449,.439,10,0,0,0,NaN,True
4,Chicago Bulls,/teams/CHI/,NBA,1966-67,2022-23,57,4551,2320,2231,.510,36,9,6,6,NaN,True
5,Cleveland Cavaliers,/teams/CLE/,NBA,1970-71,2022-23,53,4225,1955,2270,.463,22,7,5,1,NaN,True
6,Dallas Mavericks,/teams/DAL/,NBA,1980-81,2022-23,43,3415,1729,1686,.506,24,4,2,1,NaN,True
7,Denver Nuggets,/teams/DEN/,NBA/ABA,1967-68,2022-23,56,4484,2280,2204,.508,37,12,0,0,NaN,True
8,Detroit Pistons,/teams/DET/,NBA/BAA,1948-49,2022-23,75,5872,2805,3067,.478,42,11,5,3,NaN,True
9,Golden State Warriors,/teams/GSW/,NBA/BAA,1946-47,2022-23,77,5976,2898,3078,.485,36,12,7,7,NaN,True


All Season Summary

In [ ]:
def parse_all_franchise_seasons(soup):
    team_table = parse_table(soup,f'#content tbody tr')
    team_table['playoffs'] = team_table['team_name'].str.endswith("*")
    team_table['team_name'] = team_table['team_name'].str.replace("*","",regex=False)
    return team_table

for i,franchise_i in all_franchises.iterrows():
    host = f'basketball-reference.com'
    url = f"{host}{franchise_i['franch_name_href']}"
    soup = BeautifulSoup(fetch_html(f"{url}"))
    all_franchise_seasons = parse_all_franchise_seasons(soup)
    if not os.path.isdir(f"{url}"): 
        os.makedirs(f"{url}")
    # all_franchise_seasons.to_csv(f"{url}/index.csv")
    print(f"{url}/index.csv")
    time.sleep(3.2)

In [ ]:
all_franchise_seasons = pd.read_csv('basketball-reference.com/teams/GSW/index.csv').drop(columns='Unnamed: 0')
all_franchise_seasons

,season,season_href,lg_id,lg_id_href,team_name,team_name_href,wins,losses,win_loss_pct,rank_team,...,off_rtg,off_rtg_rel,def_rtg,def_rtg_rel,rank_team_playoffs,coaches,coaches_href,top_ws,top_ws_href,playoffs
0,2022-23,/teams/GSW/2023.html,NBA,/leagues/NBA_2023.html,Golden State Warriors,/teams/GSW/2023.html,18,18,0.500,4th of 5,...,113.1,-0.3,113.7,0.3,NaN,S. Kerr (18-18),/coaches/kerrst01c.html,S. Curry (4.1),/players/c/curryst01.html,False
1,2021-22,/teams/GSW/2022.html,NBA,/leagues/NBA_2022.html,Golden State Warriors,/teams/GSW/2022.html,53,29,0.646,2nd of 5,...,112.5,0.5,106.9,-5.1,Won Finals,S. Kerr (53-29),/coaches/kerrst01c.html,S. Curry (8.0),/players/c/curryst01.html,True
2,2020-21,/teams/GSW/2021.html,NBA,/leagues/NBA_2021.html,Golden State Warriors,/teams/GSW/2021.html,39,33,0.542,4th of 5,...,111.1,-1.2,110.1,-2.2,NaN,S. Kerr (39-33),/coaches/kerrst01c.html,S. Curry (9.0),/players/c/curryst01.html,False
3,2019-20,/teams/GSW/2020.html,NBA,/leagues/NBA_2020.html,Golden State Warriors,/teams/GSW/2020.html,15,50,0.231,5th of 5,...,105.2,-5.4,113.8,3.2,NaN,S. Kerr (15-50),/coaches/kerrst01c.html,M. Chriss (3.4),/players/c/chrisma01.html,False
4,2018-19,/teams/GSW/2019.html,NBA,/leagues/NBA_2019.html,Golden State Warriors,/teams/GSW/2019.html,57,25,0.695,1st of 5,...,115.9,5.5,109.5,-0.9,Lost Finals,S. Kerr (57-25),/coaches/kerrst01c.html,K. Durant (11.5),/players/d/duranke01.html,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1950-51,/teams/PHW/1951.html,NBA,/leagues/NBA_1951.html,Philadelphia Warriors,/teams/PHW/1951.html,40,26,0.606,1st of 6,...,84.8,-0.3,81.0,-4.1,Lost E. Div. Semis,E. Gottlieb (40-26),/coaches/gottled99c.html,P. Arizin (13.7),/players/a/arizipa01.html,True
73,1949-50,/teams/PHW/1950.html,NBA,/leagues/NBA_1950.html,Philadelphia Warriors,/teams/PHW/1950.html,26,42,0.382,4th of 6,...,NaN,NaN,NaN,NaN,Lost E. Div. Semis,E. Gottlieb (26-42),/coaches/gottled99c.html,L. Mogus (8.0),/players/m/mogusle01.html,True
74,1948-49,/teams/PHW/1949.html,BAA,/leagues/BAA_1949.html,Philadelphia Warriors,/teams/PHW/1949.html,28,32,0.467,4th of 6,...,NaN,NaN,NaN,NaN,Lost E. Div. Semis,E. Gottlieb (28-32),/coaches/gottled99c.html,E. Sadowski (10.7),/players/s/sadowed01.html,True
75,1947-48,/teams/PHW/1948.html,BAA,/leagues/BAA_1948.html,Philadelphia Warriors,/teams/PHW/1948.html,27,21,0.563,1st of 4,...,NaN,NaN,NaN,NaN,Lost Finals,E. Gottlieb (27-21),/coaches/gottled99c.html,H. Dallmar (7.0),/players/d/dallmho01.html,True


Season Game Logs

In [ ]:
def parse_season_gamelogs_basic(soup):
    team_table_regular = parse_table(soup,f'#tgl_basic > tbody > tr:not(.thead)')
    team_table_playoffs = parse_table(soup,f'#tgl_basic_playoffs > tbody > tr:not(.thead)')
    return team_table_regular,team_table_playoffs

def parse_season_gamelogs_advanced(soup):
    team_table_regular = parse_table(soup,f'#tgl_advanced > tbody > tr:not(.thead)')
    team_table_playoffs = parse_table(soup,f'#tgl_advanced_playoffs > tbody > tr:not(.thead)')
    return team_table_regular,team_table_playoffs

for i,franchise_i in all_franchises.iterrows():
    url_i = f"{host}{franchise_i['franch_name_href']}"
    franchise_i_seasons = pd.read_csv(f'{url_i}index.csv').drop(columns='Unnamed: 0')

    for j,season_j in franchise_i_seasons.iterrows():
        for gl_type in ['gamelog','gamelog-advanced']:
            url_j = f"{host}{season_j['season_href'].strip('.html')}/{gl_type}"
            print(f"{url_j}")
            if not os.path.isdir(f"{url_j}"): 
                os.makedirs(f"{url_j}")
                soup = BeautifulSoup(fetch_html(f"{url_j}"))
                if gl_type == 'gamelog':
                    season_j_gamelogs_regular, season_j_gamelogs_playoffs = parse_season_gamelogs_basic(soup)
                else:
                    season_j_gamelogs_regular, season_j_gamelogs_playoffs = parse_season_gamelogs_advanced(soup)

                if not season_j_gamelogs_regular.empty:
                    season_j_gamelogs_regular.to_csv(f"{url_j}/regular.csv")
                if not season_j_gamelogs_playoffs.empty:
                    season_j_gamelogs_playoffs.to_csv(f"{url_j}/playoffs.csv")
                time.sleep(3.2)


Season Game Logs Prior

Basic Stats

In [7]:
def find_season_urls(team=''):
    all_franchises = pd.read_csv(f"basketball-reference.com/teams/index.csv")
    franchises = all_franchises[all_franchises['franch_name_href'].str.contains(team)]
    host = f'basketball-reference.com'
    urls = []
    for i,franchise_i in franchises.iterrows():
        url_i = f"{host}{franchise_i['franch_name_href']}"
        franchise_i_seasons = pd.read_csv(f'{url_i}index.csv').drop(columns='Unnamed: 0')
        for j,season_j in franchise_i_seasons.iterrows():
            url_j = f"{season_j['season_href']}"
            urls.append(url_j)
    return urls


In [ ]:

def shift_values(x,n=1,pad=np.nan):
    if isinstance(x,pd.Series):
        x.loc[:] = [pad]*n + list(x.values[:-n])
    elif isinstance(x,pd.DataFrame):
        x = pd.concat([pd.DataFrame(columns=x.columns,index=range(n)),x.iloc[:-n]],ignore_index=True)
    return x


for season_j in find_season_urls():
    url_j = f"{host}{season_j.strip('.html')}"
    print(url_j)

    if not os.path.isdir(f"{url_j}/gamelog-prior"):
        os.makedirs(f"{url_j}/gamelog-prior")

    team_basic = pd.read_csv(f'{url_j}/gamelog/regular.csv').drop(columns=['Unnamed: 0','ranker','x'])
    team_advanced = pd.read_csv(f'{url_j}/gamelog-advanced/regular.csv').drop(columns=['Unnamed: 0','ranker','x'])

    team_basic.loc[:,'game_location'] = team_basic['game_location'] != '@'
    team_basic.loc[:,'game_result'] = team_basic['game_result'] == 'W'
    team_basic.loc[:,'net_pts'] = team_basic['pts'] - team_basic['opp_pts']
    team_advanced.loc[:,'net_rtg'] = team_advanced['off_rtg'] - team_advanced['def_rtg']

    # Game info
    game_info = team_basic.columns[:7]
    team_prior = team_basic.loc[:,game_info].copy()
    team_prior.insert(2,"team_id",season_j[7:10])
    team_prior.insert(3,"team_id_href",season_j)
    team_prior['rest_days_prior'] = pd.to_datetime(team_basic['date_game']).diff().apply(lambda x: x/np.timedelta64(1, 'D')).fillna(0).astype('int64')
    # team_prior = team_prior

    # Total/H/A/H2H W%
    home_record_prior   = shift_values(team_basic.loc[(team_basic['game_location']),'game_result'].expanding().mean())
    away_record_prior   = shift_values(team_basic.loc[(~team_basic['game_location']),'game_result'].expanding().mean())
    team_prior.loc[:,'result_prior'] =  shift_values(team_basic['game_result'].expanding().mean())
    team_prior.loc[:,'location_result_prior'] = pd.concat([home_record_prior,away_record_prior]).sort_index()
    for opp in pd.unique(team_prior['opp_id']):
        team_prior.loc[(team_prior['opp_id']==opp),'h2h_result_prior'] = shift_values(team_basic['game_result'][team_basic['opp_id']==opp].expanding().mean())

    # Team Basic & Advanced Stats
    basic_stats = team_basic.columns[7:]
    advanced_stats = team_advanced.columns[9:]
    team_prior.loc[:,basic_stats] = shift_values(team_basic.loc[:,basic_stats].expanding().mean())
    team_prior.loc[:,advanced_stats] = shift_values(team_advanced.loc[:,advanced_stats].expanding().mean())

    # L10 Stats (Delta)
    team_prior.loc[:,'result_last10_delta'] =  shift_values(team_basic['game_result'].rolling(10).mean())
    team_prior = team_prior.join(shift_values(team_basic.loc[:,basic_stats].rolling(10).mean()) - team_prior.loc[:,basic_stats],rsuffix="_last10_delta")
    team_prior = team_prior.join(shift_values(team_advanced.loc[:,advanced_stats].rolling(10).mean()) - team_prior.loc[:,advanced_stats],rsuffix="_last10_delta")

    team_prior.to_csv(f'{url_j}/gamelog-prior/regular.csv')
    time.sleep(5)
    # team_prior.head(30)

In [145]:
team_basic = pd.read_csv('basketball-reference.com/teams/DAL/2011/gamelog/regular.csv').drop(columns="Unnamed: 0")
team_basic.loc[:,'game_location'] = team_basic['game_location'] != '@'
team_basic.loc[:,'game_result'] = team_basic['game_result'] == 'W'
team_basic = team_basic.drop(columns='ranker')
print(team_basic.columns)
team_basic

Index(['game_season', 'date_game', 'date_game_href', 'game_location', 'opp_id',
       'opp_id_href', 'game_result', 'pts', 'opp_pts', 'fg', 'fga', 'fg_pct',
       'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'trb', 'ast',
       'stl', 'blk', 'tov', 'pf', 'x', 'opp_fg', 'opp_fga', 'opp_fg_pct',
       'opp_fg3', 'opp_fg3a', 'opp_fg3_pct', 'opp_ft', 'opp_fta', 'opp_ft_pct',
       'opp_orb', 'opp_trb', 'opp_ast', 'opp_stl', 'opp_blk', 'opp_tov',
       'opp_pf'],
      dtype='object')


,game_season,date_game,date_game_href,game_location,opp_id,opp_id_href,game_result,pts,opp_pts,fg,...,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf
0,1,2010-10-27,/boxscores/201010270DAL.html,True,CHA,/teams/CHA/2011.html,True,101,86,41,...,23,24,0.958,8,36,22,8,3,20,15
1,2,2010-10-29,/boxscores/201010290DAL.html,True,MEM,/teams/MEM/2011.html,False,90,91,34,...,25,36,0.694,14,45,17,12,3,13,24
2,3,2010-10-31,/boxscores/201010310LAC.html,False,LAC,/teams/LAC/2011.html,True,99,83,36,...,23,31,0.742,15,48,17,7,3,16,23
3,4,2010-11-03,/boxscores/201011030DEN.html,False,DEN,/teams/DEN/2011.html,True,102,101,39,...,9,12,0.750,13,39,25,7,3,11,21
4,5,2010-11-06,/boxscores/201011060DAL.html,True,DEN,/teams/DEN/2011.html,False,92,103,34,...,16,21,0.762,9,38,21,16,2,12,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2011-04-06,/boxscores/201104060DAL.html,True,DEN,/teams/DEN/2011.html,False,96,104,39,...,14,18,0.778,12,40,23,9,4,14,17
78,79,2011-04-08,/boxscores/201104080DAL.html,True,LAC,/teams/LAC/2011.html,True,107,96,43,...,14,20,0.700,9,36,22,12,6,21,23
79,80,2011-04-10,/boxscores/201104100DAL.html,True,PHO,/teams/PHO/2011.html,True,115,90,45,...,18,33,0.545,14,37,19,5,1,14,18
80,81,2011-04-11,/boxscores/201104110HOU.html,False,HOU,/teams/HOU/2011.html,True,98,91,38,...,13,16,0.813,17,52,24,11,1,17,19


In [146]:
'/teams/CHA/2011.html'[7:10]

'CHA'

In [141]:
# team_basic.insert(1,"team_id","DAL")
team_basic

,game_season,team_id,date_game,date_game_href,game_location,opp_id,opp_id_href,game_result,pts,opp_pts,...,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf
0,1,DAL,2010-10-27,/boxscores/201010270DAL.html,True,CHA,/teams/CHA/2011.html,True,101,86,...,23,24,0.958,8,36,22,8,3,20,15
1,2,DAL,2010-10-29,/boxscores/201010290DAL.html,True,MEM,/teams/MEM/2011.html,False,90,91,...,25,36,0.694,14,45,17,12,3,13,24
2,3,DAL,2010-10-31,/boxscores/201010310LAC.html,False,LAC,/teams/LAC/2011.html,True,99,83,...,23,31,0.742,15,48,17,7,3,16,23
3,4,DAL,2010-11-03,/boxscores/201011030DEN.html,False,DEN,/teams/DEN/2011.html,True,102,101,...,9,12,0.750,13,39,25,7,3,11,21
4,5,DAL,2010-11-06,/boxscores/201011060DAL.html,True,DEN,/teams/DEN/2011.html,False,92,103,...,16,21,0.762,9,38,21,16,2,12,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,DAL,2011-04-06,/boxscores/201104060DAL.html,True,DEN,/teams/DEN/2011.html,False,96,104,...,14,18,0.778,12,40,23,9,4,14,17
78,79,DAL,2011-04-08,/boxscores/201104080DAL.html,True,LAC,/teams/LAC/2011.html,True,107,96,...,14,20,0.700,9,36,22,12,6,21,23
79,80,DAL,2011-04-10,/boxscores/201104100DAL.html,True,PHO,/teams/PHO/2011.html,True,115,90,...,18,33,0.545,14,37,19,5,1,14,18
80,81,DAL,2011-04-11,/boxscores/201104110HOU.html,False,HOU,/teams/HOU/2011.html,True,98,91,...,13,16,0.813,17,52,24,11,1,17,19


In [ ]:
team_advanced = pd.read_csv('basketball-reference.com/teams/DAL/2011/gamelog-advanced/regular.csv').drop(columns="Unnamed: 0")
print(team_advanced.columns)
team_advanced

Index(['ranker', 'game_season', 'date_game', 'date_game_href', 'game_location',
       'opp_id', 'opp_id_href', 'game_result', 'pts', 'opp_pts', 'off_rtg',
       'def_rtg', 'pace', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'ts_pct',
       'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'x', 'efg_pct', 'tov_pct',
       'orb_pct', 'ft_rate', 'opp_efg_pct', 'opp_tov_pct', 'drb_pct',
       'opp_ft_rate'],
      dtype='object')


,ranker,game_season,date_game,date_game_href,game_location,opp_id,opp_id_href,game_result,pts,opp_pts,...,blk_pct,x,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate
0,1,1,2010-10-27,/boxscores/201010270DAL.html,NaN,CHA,/teams/CHA/2011.html,W,101,86,...,8.3,NaN,0.573,16.2,20.0,0.200,0.432,19.3,81.0,0.315
1,2,2,2010-10-29,/boxscores/201010290DAL.html,NaN,MEM,/teams/MEM/2011.html,L,90,91,...,9.2,NaN,0.514,17.9,20.5,0.222,0.413,11.9,72.5,0.313
2,3,3,2010-10-31,/boxscores/201010310LAC.html,@,LAC,/teams/LAC/2011.html,W,99,83,...,11.5,NaN,0.513,13.5,19.5,0.260,0.366,14.3,68.8,0.280
3,4,4,2010-11-03,/boxscores/201011030DEN.html,@,DEN,/teams/DEN/2011.html,W,102,101,...,8.2,NaN,0.506,12.3,31.6,0.202,0.517,10.4,71.7,0.101
4,5,5,2010-11-06,/boxscores/201011060DAL.html,NaN,DEN,/teams/DEN/2011.html,L,92,103,...,8.2,NaN,0.474,19.4,27.5,0.263,0.530,11.6,79.5,0.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,78,2011-04-06,/boxscores/201104060DAL.html,NaN,DEN,/teams/DEN/2011.html,L,96,104,...,14.3,NaN,0.519,15.3,34.9,0.163,0.523,13.0,71.4,0.163
78,79,79,2011-04-08,/boxscores/201104080DAL.html,NaN,LAC,/teams/LAC/2011.html,W,107,96,...,6.9,NaN,0.529,14.9,35.7,0.200,0.539,19.8,75.0,0.184
79,80,80,2011-04-10,/boxscores/201104100DAL.html,NaN,PHO,/teams/PHO/2011.html,W,115,90,...,6.5,NaN,0.567,12.6,43.9,0.157,0.439,12.7,70.2,0.220
80,81,81,2011-04-11,/boxscores/201104110HOU.html,@,HOU,/teams/HOU/2011.html,W,98,91,...,7.2,NaN,0.482,17.9,14.6,0.217,0.406,14.2,71.7,0.135


Table prior

In [ ]:
def shift_values(x,n=1,pad=np.nan):
    if isinstance(x,pd.Series):
        x.loc[:] = [pad]*n + list(x.values[:-n])
    elif isinstance(x,pd.DataFrame):
        x = pd.concat([pd.DataFrame(columns=x.columns,index=range(n)),x.iloc[:-n]],ignore_index=True)
    return x


# Game info
team_prior = team_basic.iloc[:,:6].copy()
team_prior['rest_days_prior'] = pd.to_datetime(team_basic['date_game']).diff().apply(lambda x: x/np.timedelta64(1, 'D')).fillna(0).astype('int64')

# Total/H/A/H2H W%
home_record_prior   = shift_values(team_basic.loc[(team_basic['game_location']),'game_result'].expanding().mean())
away_record_prior   = shift_values(team_basic.loc[(~team_basic['game_location']),'game_result'].expanding().mean())
team_prior.loc[:,'result_prior'] =  shift_values(team_basic['game_result'].expanding().mean())
team_prior.loc[:,'location_result_prior'] = pd.concat([home_record_prior,away_record_prior]).sort_index()
for opp in pd.unique(team_prior['opp_id']):
    team_prior.loc[(team_prior['opp_id']==opp),'h2h_result_prior'] = shift_values(team_basic['game_result'][team_basic['opp_id']==opp].expanding().mean())

# Team Basic & Advanced Stats
basic_stats = team_basic.columns[7:].drop('x')
advanced_stats = team_advanced.columns[10:].drop('x')
team_prior.loc[:,basic_stats] = shift_values(team_basic.loc[:,basic_stats].expanding().mean())
team_prior.loc[:,advanced_stats] = shift_values(team_advanced.loc[:,advanced_stats].expanding().mean())

# L10 Stats (Delta)
team_prior.loc[:,'result_last10_delta'] =  shift_values(team_basic['game_result'].rolling(10).mean())
team_prior = team_prior.join(shift_values(team_basic.loc[:,basic_stats].rolling(10).mean()) - team_prior.loc[:,basic_stats],rsuffix="_last10_delta")
team_prior = team_prior.join(shift_values(team_advanced.loc[:,advanced_stats].rolling(10).mean()) - team_prior.loc[:,advanced_stats],rsuffix="_last10_delta")

# team_prior.loc[:,advanced_stats+'last10_delta'] = shift_values(team_advanced.loc[:,advanced_stats].rolling(10).mean()) - team_prior.loc[:,advanced_stats]
print(team_prior.columns.values)
team_prior.head(30)

['game_season' 'date_game' 'date_game_href' 'game_location' 'opp_id'
 'opp_id_href' 'rest_days_prior' 'result_prior' 'location_result_prior'
 'h2h_result_prior' 'pts' 'opp_pts' 'fg' 'fga' 'fg_pct' 'fg3' 'fg3a'
 'fg3_pct' 'ft' 'fta' 'ft_pct' 'orb' 'trb' 'ast' 'stl' 'blk' 'tov' 'pf'
 'opp_fg' 'opp_fga' 'opp_fg_pct' 'opp_fg3' 'opp_fg3a' 'opp_fg3_pct'
 'opp_ft' 'opp_fta' 'opp_ft_pct' 'opp_orb' 'opp_trb' 'opp_ast' 'opp_stl'
 'opp_blk' 'opp_tov' 'opp_pf' 'off_rtg' 'def_rtg' 'pace' 'fta_per_fga_pct'
 'fg3a_per_fga_pct' 'ts_pct' 'trb_pct' 'ast_pct' 'stl_pct' 'blk_pct'
 'efg_pct' 'tov_pct' 'orb_pct' 'ft_rate' 'opp_efg_pct' 'opp_tov_pct'
 'drb_pct' 'opp_ft_rate' 'result_last10_delta' 'pts_last10_delta'
 'opp_pts_last10_delta' 'fg_last10_delta' 'fga_last10_delta'
 'fg_pct_last10_delta' 'fg3_last10_delta' 'fg3a_last10_delta'
 'fg3_pct_last10_delta' 'ft_last10_delta' 'fta_last10_delta'
 'ft_pct_last10_delta' 'orb_last10_delta' 'trb_last10_delta'
 'ast_last10_delta' 'stl_last10_delta' 'blk_last10_de

,game_season,date_game,date_game_href,game_location,opp_id,opp_id_href,rest_days_prior,result_prior,location_result_prior,h2h_result_prior,...,stl_pct_last10_delta,blk_pct_last10_delta,efg_pct_last10_delta,tov_pct_last10_delta,orb_pct_last10_delta,ft_rate_last10_delta,opp_efg_pct_last10_delta,opp_tov_pct_last10_delta,drb_pct_last10_delta,opp_ft_rate_last10_delta
0,1,2010-10-27,/boxscores/201010270DAL.html,True,CHA,/teams/CHA/2011.html,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2010-10-29,/boxscores/201010290DAL.html,True,MEM,/teams/MEM/2011.html,2,1.000000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2010-10-31,/boxscores/201010310LAC.html,False,LAC,/teams/LAC/2011.html,2,0.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2010-11-03,/boxscores/201011030DEN.html,False,DEN,/teams/DEN/2011.html,3,0.666667,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2010-11-06,/boxscores/201011060DAL.html,True,DEN,/teams/DEN/2011.html,3,0.750000,0.500000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2010-11-08,/boxscores/201011080DAL.html,True,BOS,/teams/BOS/2011.html,2,0.600000,0.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2010-11-10,/boxscores/201011100MEM.html,False,MEM,/teams/MEM/2011.html,2,0.666667,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,2010-11-12,/boxscores/201011120DAL.html,True,PHI,/teams/PHI/2011.html,2,0.714286,0.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,2010-11-15,/boxscores/201011150DAL.html,True,NOH,/teams/NOH/2011.html,3,0.750000,0.600000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2010-11-17,/boxscores/201011170NOH.html,False,NOH,/teams/NOH/2011.html,2,0.777778,1.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merge Team with Opp Prior stats

In [5]:
team = pd.read_csv('basketball-reference.com/teams/GSW/2022/gamelog/regular.csv').drop(columns="Unnamed: 0")
opp = pd.read_csv('basketball-reference.com/teams/GSW/2022/gamelog-advanced/regular.csv').drop(columns="Unnamed: 0")
team.merge(opp,how='inner',on='date_game_href',suffixes=['','_advanced'])

,ranker,game_season,date_game,date_game_href,game_location,opp_id,opp_id_href,game_result,pts,opp_pts,...,blk_pct,x_advanced,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate
0,1,1,2021-10-19,/boxscores/202110190LAL.html,@,LAL,/teams/LAL/2022.html,W,121,114,...,3.8,NaN,0.516,13.8,18.4,0.269,0.553,14.1,89.1,0.095
1,2,2,2021-10-21,/boxscores/202110210GSW.html,NaN,LAC,/teams/LAC/2022.html,W,115,113,...,7.1,NaN,0.608,18.5,24.4,0.169,0.510,6.2,84.3,0.156
2,3,3,2021-10-24,/boxscores/202110240SAC.html,@,SAC,/teams/SAC/2022.html,W,119,107,...,7.5,NaN,0.510,5.3,27.5,0.206,0.576,17.2,74.4,0.093
3,4,4,2021-10-26,/boxscores/202110260OKC.html,@,OKC,/teams/OKC/2022.html,W,106,98,...,15.4,NaN,0.554,14.2,17.5,0.169,0.488,12.2,81.3,0.163
4,5,5,2021-10-28,/boxscores/202110280GSW.html,NaN,MEM,/teams/MEM/2022.html,L,101,104,...,14.3,NaN,0.511,18.8,14.6,0.100,0.464,14.1,80.4,0.144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,78,2022-04-02,/boxscores/202204020GSW.html,NaN,UTA,/teams/UTA/2022.html,W,111,107,...,14.0,NaN,0.595,11.0,14.6,0.131,0.511,10.8,74.5,0.180
78,79,79,2022-04-03,/boxscores/202204030SAC.html,@,SAC,/teams/SAC/2022.html,W,109,90,...,16.7,NaN,0.620,14.9,19.4,0.139,0.426,11.6,87.0,0.259
79,80,80,2022-04-07,/boxscores/202204070GSW.html,NaN,LAL,/teams/LAL/2022.html,W,128,112,...,9.8,NaN,0.608,12.9,20.9,0.103,0.500,15.3,81.8,0.349
80,81,81,2022-04-09,/boxscores/202204090SAS.html,@,SAS,/teams/SAS/2022.html,W,100,94,...,2.0,NaN,0.457,11.2,26.0,0.321,0.463,11.6,90.5,0.250


In [175]:
for season_j in find_season_urls():
    url_j = f"{host}{season_j.strip('.html')}"
    print(url_j)

    if not os.path.isdir(f"{folder}{url_j}/gamelog-prior-all") and os.path.isfile(f'{folder}{url_j}/gamelog-prior/regular.csv'):
        os.makedirs(f"{folder}{url_j}/gamelog-prior-all")
        team_j_prior = pd.read_csv(f'{folder}{url_j}/gamelog-prior/regular.csv').drop(columns="Unnamed: 0")
        team_j_prior_all = pd.DataFrame()
        for opp_k in pd.unique(team_j_prior['opp_id_href']):
            url_k = f"{host}{opp_k.strip('.html')}"
            opp_k_prior = pd.read_csv(f'{folder}{url_k}/gamelog-prior/regular.csv').drop(columns="Unnamed: 0")
            team_j_opp_k_prior = team_j_prior.merge(opp_k_prior,how='inner',on='date_game_href',suffixes=['_teamA','_teamB'])
            team_j_prior_all = pd.concat([team_j_prior_all,team_j_opp_k_prior],ignore_index=True).sort_values(by='game_season_teamA')
            team_j_prior_all.to_csv(f'{folder}{url_j}/gamelog-prior-all/regular.csv')

    

basketball-reference.com/teams/ATL/2023
basketball-reference.com/teams/ATL/2022
basketball-reference.com/teams/ATL/2021
basketball-reference.com/teams/ATL/2020
basketball-reference.com/teams/ATL/2019


In [187]:
# if not os.path.isdir(f"{folder}data/gamelog-prior-all"):
#     os.makedirs(f"{folder}data/gamelog-prior-all")

def write(content,filepath,mode='w'):
    filedir,filename = filedir_name(filepath)
    if not os.path.isdir(filedir): 
        os.makedirs(filedir)
    with open(filepath, mode) as file:
        file.write(content)

def read_csv_as_str(filepath,skip_header=False):
    content = ""
    with open(filepath, 'r') as file:
        if skip_header:
            file.readline()
        content = file.read()
    return content


for j,season_j in enumerate(find_season_urls()[:5]):
    url_j = f"{host}{season_j.strip('.html')}"
    print(url_j)

    infile = f"{folder}{url_j}/gamelog-prior-all/regular.csv"
    outfile = f"{folder}data/gamelog-prior-all/regular_demo.csv"
    if j == 0:
        write(read_csv_as_str(infile,skip_header=False),outfile,'w')
    else:
        write(read_csv_as_str(infile,skip_header=True),outfile,'a')

basketball-reference.com/teams/ATL/2023
basketball-reference.com/teams/ATL/2022
basketball-reference.com/teams/ATL/2021
basketball-reference.com/teams/ATL/2020
basketball-reference.com/teams/ATL/2019


In [12]:
for j,season_j in enumerate(find_season_urls()[:5]):
    url_j = f"{host}{season_j.strip('.html')}"
    print(url_j)

    infile_basic = f"{folder}{url_j}/gamelog/regular.csv"
    infile_advanced = f"{folder}{url_j}/gamelog-advanced/regular.csv"
    temp_file = f"{folder}{url_j}/gamelog-all/regular.csv"
    outfile = f"{folder}data/gamelog-all/regular_demo.csv"



    if os.path.isfile(infile_basic) and os.path.isfile(infile_advanced):
        if not os.path.isdir(f"{folder}{url_j}/gamelog-all"):
            os.makedirs(f"{folder}{url_j}/gamelog-all")
        gamelog_basic = pd.read_csv(infile_basic).drop(columns=['Unnamed: 0','ranker','x'])
        gamelog_advanced = pd.read_csv(infile_advanced).drop(columns=['Unnamed: 0','ranker','x'])
        gamelog_all = gamelog_basic.merge(gamelog_advanced,how='inner',on='date_game_href',suffixes=['','_advanced'])
        gamelog_all.insert(2,"team_id",season_j[7:10])
        gamelog_all.insert(3,"team_id_href",season_j)
        gamelog_all['net_pts'] = gamelog_all['pts'] - gamelog_all['opp_pts']
        gamelog_all['net_rtg'] = gamelog_all['off_rtg'] - gamelog_all['def_rtg']
        gamelog_all.to_csv(temp_file)
        
        if j == 0:
            write(read_csv_as_str(temp_file,skip_header=False),outfile,'w')
        else:
            write(read_csv_as_str(temp_file,skip_header=True),outfile,'a')


basketball-reference.com/teams/ATL/2023
basketball-reference.com/teams/ATL/2022
basketball-reference.com/teams/ATL/2021
basketball-reference.com/teams/ATL/2020
basketball-reference.com/teams/ATL/2019


In [3]:
all_gamelog_prior = pd.read_csv(f"{folder}data/gamelog-prior-all/regular.csv").drop(columns="Unnamed: 0")
all_gamelog_prior

,game_season_teamA,date_game_teamA,team_id_teamA,team_id_href_teamA,date_game_href,game_location_teamA,opp_id_teamA,opp_id_href_teamA,game_result_teamA,rest_days_prior_teamA,...,blk_pct_last10_delta_teamB,efg_pct_last10_delta_teamB,tov_pct_last10_delta_teamB,orb_pct_last10_delta_teamB,ft_rate_last10_delta_teamB,opp_efg_pct_last10_delta_teamB,opp_tov_pct_last10_delta_teamB,drb_pct_last10_delta_teamB,opp_ft_rate_last10_delta_teamB,net_rtg_last10_delta_teamB
0,1,2022-10-19,ATL,/teams/ATL/2023.html,/boxscores/202210190ATL.html,True,HOU,/teams/HOU/2023.html,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2022-10-21,ATL,/teams/ATL/2023.html,/boxscores/202210210ATL.html,True,ORL,/teams/ORL/2023.html,True,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2022-10-23,ATL,/teams/ATL/2023.html,/boxscores/202210230ATL.html,True,CHO,/teams/CHO/2023.html,False,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2022-10-26,ATL,/teams/ATL/2023.html,/boxscores/202210260DET.html,False,DET,/teams/DET/2023.html,True,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2022-10-28,ATL,/teams/ATL/2023.html,/boxscores/202210280DET.html,False,DET,/teams/DET/2023.html,True,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135877,58,1950-03-01,WAT,/teams/WAT/1950.html,/boxscores/195003010WAT.html,True,SYR,/teams/SYR/1950.html,False,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135878,59,1950-03-08,WAT,/teams/WAT/1950.html,/boxscores/195003080WAT.html,True,DNN,/teams/DNN/1950.html,True,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135879,60,1950-03-12,WAT,/teams/WAT/1950.html,/boxscores/195003120INO.html,False,INO,/teams/INO/1950.html,False,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135880,61,1950-03-15,WAT,/teams/WAT/1950.html,/boxscores/195003150WAT.html,True,SHE,/teams/SHE/1950.html,True,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
